In [6]:
import pandas as pd
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score
import numpy as np
import matplotlib.pyplot as plt

df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

#Buscar simple Inputer


Index(['id', 'titulo', 'descripcion', 'tipodepropiedad', 'direccion', 'ciudad',
       'provincia', 'antiguedad', 'habitaciones', 'garages', 'banos',
       'metroscubiertos', 'metrostotales', 'idzona', 'lat', 'lng', 'fecha',
       'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas',
       'centroscomercialescercanos', 'precio'],
      dtype='object')
Index(['id', 'titulo', 'descripcion', 'tipodepropiedad', 'direccion', 'ciudad',
       'provincia', 'antiguedad', 'habitaciones', 'garages', 'banos',
       'metroscubiertos', 'metrostotales', 'idzona', 'lat', 'lng', 'fecha',
       'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas',
       'centroscomercialescercanos'],
      dtype='object')
876
577
33
33
['Apartamento' 'Casa en condominio' 'Casa' 'Terreno' 'Terreno comercial'
 'Local Comercial' 'Quinta Vacacional' 'Oficina comercial' 'Edificio'
 'Casa uso de suelo' 'Local en centro comercial' 'Bodega comercial'
 'Otros' 'Villa' 'Duplex' 'Inmuebles productivos urbanos'
 

ValueError: operands could not be broadcast together with shapes (25,) (23,) 

In [38]:

#precio = df_train_xgboost['precio']
#df_train_xgboost.drop(labels=['precio'], axis=1,inplace = True)
#df_train_xgboost.insert(len(df_train_xgboost.columns), 'Precio', precio)
ciudades_list = ['Benito Juárez', 'La Magdalena Contreras', 'Tonalá',
       'Zinacantepec', 'Zapopan', 'Coyoacán', 'Oaxaca de Juárez',
       'Playa del Carmen', 'Villa de Alvarez', 'Ixtapaluca',
       'San Luis Potosí', 'San Pedro Garza García', 'Querétaro',
       'Naucalpan de Juárez', 'Monterrey', 'Ciudad Madero', 'Cancún',
       'Apodaca', 'Puebla', 'Miguel Hidalgo', 'Mérida', 'Huixquilucan',
       'Emiliano Zapata', 'Tampico', 'Atizapán de Zaragoza', 'Tlalpan',
       'Acapulco de Juárez', 'Iztacalco', 'Villahermosa',
       'Cuautitlán Izcalli', 'Irapuato', 'Epazoyucan', 'Metepec',
       'Cuauhtémoc', 'Alvaro Obregón', 'Tultitlán', 'San Pedro Cholula',
       'Venustiano Carranza', 'Coacalco de Berriozábal', 'Valle de Bravo',
       'Boca del Río', 'Tlajomulco de Zúñiga', 'León',
       'San Andrés Cholula', 'Chihuahua', 'San Nicolás de los Garza',
       'Santa Catarina', 'Aguascalientes', 'Cuernavaca', 'Xochimilco',
       'Tlalnepantla de Baz', 'Tequisquiapan', 'Hermosillo',
       'Montemorelos', 'Milpa Alta', 'Gustavo A. Madero', 'Morelia',
       'Tijuana', 'Jesús María', 'Guadalajara', 'La Paz', 'Chapala',
       'Cuajimalpa de Morelos', 'Delicias', 'Iztapalapa', 'Calimaya',
       'Azcapotzalco', 'Torreón', 'Durango', 'Los Cabos', 'Pátzcuaro',
       'Caborca', 'Culiacán', 'San Luis Río Colorado', 'Tuxtla Gutiérrez',
       'Jiutepec', 'San Pedro Mixtepec -Dto. 22 -', 'Veracruz',
       'Gral. Escobedo', 'Toluca', 'Tepic', 'San Juan del Río', 'Cuautla',
       'Guanajuato', 'Tultepec', 'Mazatlán', 'Solidaridad',
       'Cuautlancingo', 'Ecatepec de Morelos', 'Pachuca',
       'Tepeji del Río de Ocampo', 'Manzanillo', 'Tecámac', 'Nacajuca',
       'Alvarado', 'Corregidora', 'Huehuetoca', 'Tepatitlán de Morelos',
       'Santiago', 'Navojoa', 'Guadalupe', 'El Marqués', 'Temixco',
       'Arteaga', 'Juárez', 'Cárdenas', 'Soledad de Graciano Sánchez',
       'Yautepec', 'Colima', 'Mexicali', 'Tlayacapan', 'Xalapa', 'Conkal',
       'Paraíso', 'Puerto Vallarta', 'Lerma', 'Ixtlahuaca', 'Ocoyucan',
       'Tlaquepaque', 'Saltillo', 'Nezahualcóyotl',
       'Santa Isabel Xiloxoxtla', 'Progreso', 'San Miguel de Allende',
        'Jocotepec', 'Tlaxcala', 'Ocotlán', 'Xochitepec', 'Comala',
       'Motul', 'Ramos Arizpe', 'Villa del Carbón', 'Navolato',
       'Mineral de la Reforma', 'Cuautitlán', 'Tláhuac', 'Nicolás Romero',
       'Temozón', 'Yauhquemecan', 'Coyame del Sotol', 'Gómez Palacio',
       'Chetumal', 'Atlixco', 'Zumpango', 'Cadereyta Jiménez', 'Centro',
       'Ensenada', 'Tulancingo de Bravo', 'Altamira', 'Xalisco', 'Ahome',
       'Tizayuca', 'Carmen', 'Coatzacoalcos', 'Ayala', 'Chimalhuacán',
       'Campeche', 'Calpulalpan', 'Gral. Zuazua', 'Tula de Allende',
       'Monclova', 'Tepotzotlán', 'Dzemul', 'Chiautempan', 'Atlatlahucan',
       'Texcoco', 'Fortín', 'Guaymas', 'Tenancingo', 'Tarímbaro',
       'Jacona', 'Zihuatanejo de Azueta', 'Tuxpan', 'Mexicaltzingo',
       'Nogales', 'Orizaba', 'Celaya', 'San Antonio la Isla',
       'San Cristóbal de las Casas', 'Apizaco', 'Yauhquemehcan',
       'San Mateo Atenco', 'Pánuco', 'García', 'Santa María Huatulco',
       'Amozoc', 'Reynosa', 'Tlalmanalco', 'Atlautla', 'Huitzilac',
       'Cajeme', 'Chalco', 'Kanasín', 'Bacalar', 'Comapa',
       'San Martín Texmelucan', 'Bahía de Banderas', 'Tochtepec',
       'Totolac', 'Jilotepec', 'Tecate', 'Santa María Colotepec',
       'Santa Cruz Tlaxcala', 'Acambay', 'El Salto', 'Atotonilco de Tula',
       'Medellín', 'Tepoztlán', 'Zacatecas', 'Castaños',
       'Ocotlán de Morelos', 'Córdoba', 'Chiquilistlán', 'Cosoleacaque',
       'Mexquitic de Carmona', 'Tapalpa', 'Amealco de Bonfil',
       'Coscomatepec', 'Huichapan', 'Nazareno Etla', 'Puerto Peñasco',
       'Uruapan', 'Ciénega de Flores', 'Atizapán', 'Nuevo Laredo',
       'Lázaro Cárdenas', 'Aldama', 'Coatepec', 'Huejotzingo',
       'Ixtapan de la Sal', 'Melchor Ocampo', 'Zitácuaro', 'Ocoyoacac',
       'Valle de Chalco Solidaridad', 'Telchac Puerto', 'Acuña',
       'Huamantla', 'Cazones de Herrera', 'Homún', 'Cihuatlán',
       'Tehuacán', 'Zacatelco', 'La Piedad', 'Berriozábal', 'otra',
       'Chilpancingo de los Bravo', 'Ocozocoautla de Espinosa', 'Tala',
       'San Gregorio Atzompa', 'Valladolid', 'Salamanca',
       'Chiapa de Corzo', 'Silao', 'San Pablo Etla', 'Tapachula', 'Lerdo',
       'Xalatlaco', 'Ixtlahuacán de los Membrillos', 'Natívitas',
       'Huasca de Ocampo', 'Santa Cruz Xoxocotlán', 'Capulhuac',
       'Allende', 'Tecomán', 'Coatepec Harinas', 'Tulum', 'Macuspana',
       'Zapotlán el Grande', 'Xico', 'Comitán de Domínguez', 'La Barca',
       'TlaltizapAn de Zapata', 'Almoloya de Juárez',
       'Dolores Hidalgo Cuna de la Independencia Nacional', 'Cozumel',
       'Colón', 'Teziutlán', 'PolotitlAn', 'Zacoalco de Torres', 'Zacapu',
       'Atlacomulco', 'Huimilpan', 'Papalotla de Xicohténcatl',
       'Banderilla', 'Mazamitla', 'Zempoala', 'Chapultepec', 'Tlaxco',
       'Tihuatlán', 'Zamora', 'Ixtacuixtla de Mariano Matamoros',
       'Chicoloapan', 'Parras', 'Teoloyucan', 'Comalcalco', 'Guasave',
       'Sabinas', 'Dzitás', 'San Jacinto Amilpas', 'Tenango del Valle',
       'Acolman', 'Pánuco de Coronado', 'Hidalgo del Parral',
       'Santa Cruz Acatepec', 'Apaseo el Grande', 'Salina Cruz',
       'José María Morelos', 'Sucilá', 'Rafael Lara Grajales',
       'Contla de Juan Cuamatzi', 'Matehuala', 'Nextlalpan', 'Coroneo',
       'Mineral del Monte', 'Temoaya', 'Abalá', 'Chicxulub Pueblo',
       'San Felipe', 'Ciudad Valles', 'Temascalapa', 'Catemaco',
       'Miacatlán', 'Coronango', 'Minatitlán', 'San José Iturbide',
       'Cuerámaro', 'Linares', 'Tolimán', 'Xonacatlán', 'Tonatico',
       'ácatlán', 'Juan C. Bonilla', 'Salinas Victoria', 'La Antigua',
       'Abasolo', 'Matamoros', 'Sayula', 'Jojutla', 'Villa de Reyes',
       'AnAhuac', 'Santiago Miahuatlán', 'Ursulo Galván', 'Malinalco',
       'Jilotzingo', 'Tepechitlán', 'Cocula', 'Magdalena Apasco',
       'Yobaín', 'Gutiérrez Zamora', 'Tetla de la Solidaridad',
       'El Llano', 'Victoria', 'Otzolotepec', 'Jocotitlán',
       'Othón P. Blanco', 'Ahuazotepec', 'Playas de Rosarito',
       'Tepetlixpa', 'Atenco', 'Calkiní',
       'ApetatitlAn de Antonio Carvajal', 'Tequixquiac',
       'Santa Cruz Amilpas', 'Teuchitlán', 'Mayapán', 'San Fernando',
       'El Espinal', 'Charo', 'Tangancícuaro', 'Quintana Roo',
       'Papalotla', 'Cosío', 'San Agustín Etla', 'Santa María del Río',
       'Cuitzeo', 'Ojinaga', 'Amaxac de Guerrero', 'Yecapixtla',
       'Coyotepec', 'Ecatzingo', 'Tianguistenco', 'Tlaltizapán', 'Otumba',
       'Pueblo Viejo', 'San Pedro Pochutla', 'Tonanitla',
       'Rincón de Romos', 'Jamapa', 'Pedro Escobedo', 'Piedras Negras',
       'Ezequiel Montes', 'El Oro', 'Atotonilco el Alto',
       'Cuautepec de Hinojosa', 'Cananea', 'Frontera', 'González',
       'Ixtlahuacán del Río', 'Jaltenco', 'Puente de Ixtla', 'Suma',
       'Sudzal', 'Santa Isabel Cholula', 'Tamuín', 'Poza Rica de Hidalgo',
       'Tianguismanalco', 'San José del Rincón', 'Jáltipan',
       'Tepetlaoxtoc', 'Ixil', 'Nanacamilpa de Mariano Arista',
       'Chapa de Mota', 'Juanacatlán', 'San Andrés Tuxtla',
       'San Sebastián Tutla', 'ácatlán de Juárez', 'Villa Purificación',
       'Morelos', 'La Huacana', 'San Bartolo Coyotepec', 'Tetecala',
       'Santa María del Tule', 'Zumpahuacán', 'San Blas', 'El Arenal',
       'Cerro Azul', 'Tampico Alto', 'Teapa', 'San Agustín de las Juntas',
       'San Martín de las Pirámides', 'Tecuala', 'Mazatepec', 'Papantla',
       'Santiago Tulantepec de Lugo Guerrero', 'Tlacotalpan',
       'Yahualica de González Gallo', 'San Pablo Villa de Mitla',
       'San Pablo del Monte', 'Panotla', 'San Miguel el Alto', 'Acatic',
       'Acajete', 'Tuxcueca', 'Jalcomulco', 'Tezoyuca', 'Hidalgo',
       'Villa Guerrero', 'Magdalena', 'Villa de Zaachila', 'Canatlán',
       'San Andrés Huayápam', 'Rioverde', 'Cerro de San Pedro',
       'Sanctórum de Lázaro Cárdenas', 'Erongarícuaro', 'Arroyo Seco',
       'Mapastepec', 'Arandas', 'Paso de Ovejas', 'Perote',
       'San Agustín Tlaxiaca', 'Chapantongo', 'Vicente Guerrero',
       'Aculco', 'Santiago Papasquiaro', 'Naranjos Amatlán', 'Tepetitlán',
       'Ixtapan del Oro', 'Jalpa de Méndez', 'Rosales', 'Tlalnepantla',
       'Tempoal', 'Zapotlán de Juárez', 'Salvatierra',
       'Atemajac de Brizuela', 'Tzompantepec', 'Balancán', 'Coatzintla',
       'Mariano Escobedo', 'Actopan', 'Tarimoro', 'Reforma',
       'Nopala de Villagrán', 'Pisaflores', 'Chiconcuac',
       'San Salvador el Verde', 'Chiautla', 'Agua Prieta', 'Cunduacán',
       'Umán', 'Jaumave', 'Santa María Atzompa', 'ácámbaro', 'Chignautla',
       'San Francisco de los Romo', 'Atoyac', 'Tepeapulco',
       'Villa de Etla', 'Acaponeta', 'Dr. Arroyo', 'Marqués de Comillas',
       'Totolapan', 'Chapulco', 'Sahuayo', 'San Ignacio',
       'Lagos de Moreno', 'Zaragoza', 'Cuetzalan del Progreso',
       'Apaseo el Alto', 'Villa Corona', 'TacAmbaro', 'Teotihuacán',
       'Ozumba', 'Los Herreras', 'Maravatío', 'Jerez', 'Alamo Temapache',
       'Jiquipilco', 'Mixquiahuala de Juárez', 'Miahuatlán', 'Teabo',
       'Tepetitla de Lardizábal', 'Ixtaczoquitlán', 'Tizimín',
       'Soto la Marina', 'Alamos', 'Nombre de Dios', 'Cocotitlán',
       'Tetela del Volcán', 'San Lorenzo Cacaotepec',
       'San Juan Bautista Tuxtepec', 'Manlio Fabio Altamirano',
       'Jonacatepec', 'Camarón de Tejeda', 'Tecozautla', 'Acayucan',
       'Izúcar de Matamoros', 'Soyaniquilpan de Juárez',
       'Cosamaloapan de Carpio', 'Compostela', 'Ameca',
       'Tlalixtac de Cabrera', 'Opichén', 'Asunción Ixtaltepec',
       'Misantla', 'Taxco de Alarcón', 'Centla', 'Empalme',
       'San Raymundo Jalpan', 'Huimanguillo', 'PuruAndiro',
       'Salvador Escalante', 'Telchac Pueblo', 'Jonuta', 'Apatzingán',
       'Tototlán', 'Temascaltepec', 'Palizada', 'Moroleón', 'Sinanché',
       'Casas', 'Palenque', 'Oluta', 'Totoltepec de Guerrero',
       'San Agustín Yatareni', 'Tepezalá', 'Fresnillo', 'Ixtlahuacán',
       'Texcaltitlán', 'Paracho', 'Río Lagartos', 'Tuxtla Chico',
       'Amecameca', 'Sinaloa', 'Villa de Arriaga', 'Cadereyta de Montes',
       'Gral. Treviño', 'Pesquería', 'Apan', 'Santa Lucía del Camino',
       'Almoloya de Alquisiras', 'Heroica Ciudad de Huajuapan de León',
       'Tecolotlán', 'Cedral', 'Acatzingo', 'Ecuandureo',
       'Villa de Tezontepec', 'Loreto', 'San Miguel Xoxtla', 'Quiroga',
       'General Cepeda', 'San Miguel de Horcasitas', 'Salvador Alvarado',
       'Teopisca', 'Hunucmá', 'Poncitlán', 'Zapotlanejo', 'Timilpan',
       'Gómez Farías', 'Los Ramones', 'Ruíz', 'Teolocholco', 'Huixtla',
       'Ucú', 'La Magdalena Tlaltelulco', 'Zinapécuaro',
       'Guadalupe Victoria', 'Villa de Allende', 'Unión de San Antonio',
       'Asientos', 'Atempan', 'ácaxochitlán',
       'Iguala de la Independencia', 'Santiago Ixcuintla', 'Champotón',
       'Hoctún', 'Bella Vista', 'Cuaxomulco', 'San Rafael', 'San Gabriel',
       'Mina', 'Celestún', 'Purísima del Rincón', 'Escárcega',
       'Villa Victoria', 'Puente Nacional', 'Huajicori',
       'Ahualulco de Mercado', 'Dzidzantún', 'Asunción Cacalotepec',
       'Cuilápam de Guerrero', 'Ixtlán del Río', 'Rafael Lucio',
       'Chilchotla', 'Zapotiltic', 'Ixhuatlancillo', 'Uriangato',
       'Camerino Z. Mendoza', 'Villaldama', 'Atitalaquia',
       'Juchitán de Zaragoza', 'Ixmiquilpan', 'Tecamachalco',
       'Encarnación de Díaz', 'Tahmek', 'La Trinitaria', 'Huauchinango',
       'Felipe Carrillo Puerto', 'Tepeyanco', 'Santa María Apazco',
       'Tetepango', 'San Joaquín', 'Uayma', 'Armería', 'Isla Mujeres',
       'Pinotepa de Don Luis', 'San Juan Bautista Guelache', 'Ahuacatlán',
       'Mineral del Chico', 'Coyuca de Benítez',
       'San Francisco del Rincón', 'Atlangatepec', 'Jiménez', 'Jiquilpan',
       'Axochiapan', 'Cotaxtla', 'Villa Corzo', 'Zacatlán', 'Villagrán',
       'El Mante', 'Cintalapa', 'Etzatlán', 'Francisco I. Madero',
       'Tepeaca', 'áyotlán', 'Aramberri', 'Tecolutla', 'Aconchi',
       'Valle Hermoso', 'Purépero', 'Amanalco', 'Jalpan de Serra',
       'Pijijiapan', 'Isidro Fabela', 'Tlahuapan', 'áutlán de Navarro',
       'Tekax', 'Casas Grandes', 'Concepción de Buenos Aires',
       'Pabellón de Arteaga', 'Petatlán', 'Apazapan',
       'Santa Cruz de Juventino Rosas', 'San Cristóbal de la Barranca',
       'Tantoyuca', 'Villa de Cos', 'Chínipas', 'Rayones', 'Jantetelco',
       'Colotlán', 'Escuinapa', 'Coquimatlán', 'Concordia', 'Comondú',
       'Zimatlán de Alvarez', 'Santa Isabel', 'Cortazar', 'Xaltocan',
       'Tixtla de Guerrero', 'Huehuetlán', 'Zacualpan', 'Tequila',
       'Huautla de Jiménez', 'Dzoncauich', 'San Francisco Lachigoló',
       'Temamatla', 'Pénjamo', 'Calera', 'Zimapán', 'Tinum',
       'Dr. González', 'Singuilucan', 'Xicotepec', 'Los Reyes', 'Timucuy',
       'Santa Ana Nopalucan', 'San Jerónimo Tecuanipan',
       'Atoyac de Alvarez', 'Chilapa de Alvarez',
       'Tlacolula de Matamoros', 'Teteles de Avila Castillo',
       'Lerdo de Tejada', 'Jerécuaro', 'Río Blanco', 'Lagunillas',
       'Chignahuapan', 'Camargo', 'Meoqui', 'Huaquechula', 'Almoloya',
       'Huetamo', 'Cacalchén', 'Gustavo Díaz Ordaz', 'Salinas',
       'Chilchota', 'Juárez Hidalgo', 'Nuevo Ideal', 'Galeana',
       'Martínez de la Torre', 'Cuzamá', 'Villaflores', 'Nahuatzen',
       'Tlatlauquitepec', 'San Francisco de Borja', 'Huandacareo',
       'Santo Domingo Tehuantepec', 'Cuautitlán de García Barragán',
       'Huatabampo', 'Suchiapa', 'San Jerónimo Tlacochahuaya',
       'Donato Guerra', 'Pichucalco', 'Tlalpujahua', 'Elota',
       'Ciudad Fernández', 'General Plutarco Elías Calles', 'Huatusco',
       'Izamal', 'Acateno', 'Mascota', 'Sabinas Hidalgo',
       'Chalchicomula de Sesma', 'Tenango del Aire', 'Tlalixcoyan',
       'Tolcayuca', 'Xaloztoc', 'Temax', 'Xicohtzinco',
       'Nanchital de Lázaro Cárdenas del Río', 'Santa Ana', 'Rosario',
       'Lamadrid', 'Axapusco', 'Amacuzac', 'Zacatepec', 'San Lucas',
       'Ajacuba', 'Teocelo', 'Contepec', 'Villanueva', 'Chavinda',
       'San Antonio de la Cal', 'El Fuerte', 'Hopelchén', 'Aquismón',
       'Almoloya del Río', 'Agua Dulce', 'San Pedro Mixtepec -Dto. 26 -',
       'Poanas', 'Calvillo', 'CopAndaro', 'Tecali de Herrera', 'Tzucacab',
       'Marín', 'Moctezuma', 'Santo Tomás', 'Angostura', 'Calcahualco',
       'Ayapango', 'Reyes Etla', 'Oriental', 'Atengo',
       'Ciénega de Zimatlán', 'Dr. Belisario Domínguez', 'Ciudad Ixtepec',
       'Jaral del Progreso', 'Higueras', 'Tzintzuntzan', 'Yuriria',
       'Coahuayana', 'Nadadores', 'Peribán', 'Saucillo',
       'Santiago Suchilquitongo', 'Tezontepec de Aldama', 'Nopaltepec',
       'Ocampo', 'Buctzotz', 'Tocumbo', 'Técpan de Galeana', 'Urique',
       'Tamiahua', 'Tamazunchale', 'Río Grande', 'Tejupilco',
       'San Felipe Orizatlán', 'Mulegé', 'Zináparo',
       'San Mateo Río Hondo', 'Ixtlán', 'Nautla', 'Jalapa',
       'Amatlán de los Reyes', 'Yaxcabá', 'San Juan Guelavía', 'Copala',
       'Batopilas', 'Acuitzio', 'Tepalcatepec', 'Atotonilco el Grande',
       'Yaxkukul', 'San Pablo Huitzo', 'Santa María del Oro',
       'Heroica Ciudad de Juchitán de Zaragoza', 'Panindícuaro',
       'Ocuilan', 'Tlaquiltenango', 'Altotonga', 'Pinal de Amoles',
       'Cansahcab', 'Ahualulco', 'Abejones', 'ZirAndaro', 'Ocosingo',
       'San Luis de la Paz', 'San Pedro', 'Gral. Bravo',
       'Progreso de Obregón', 'Bocoyna', 'El Barrio de la Soledad',
       'Omealca', 'Quitupan', 'Cerralvo', 'Sombrerete',
       'San Matías Tlalancaleca']
provincias_list = ['Distrito Federal', 'Jalisco', 'Edo. de México', 'Oaxaca',
       'Quintana Roo', 'Colima', 'San luis Potosí', 'Nuevo León',
       'Querétaro', 'Tamaulipas', 'Puebla', 'Yucatán', 'Morelos',
       'Guerrero', 'Tabasco', 'Guanajuato', 'Hidalgo', 'Veracruz',
       'Chihuahua', 'Aguascalientes', 'Sonora', 'Michoacán',
       'Baja California Norte', 'Baja California Sur', 'Coahuila',
       'Durango', 'Sinaloa', 'Chiapas', 'Nayarit', 'Tlaxcala', 'Campeche',
       'Zacatecas']

tipodepropiedad_list = ['Apartamento', 'Casa en condominio', 'Casa', 'Terreno', 'Terreno comercial'
 'Local Comercial', 'Quinta Vacacional', 'Oficina comercial', 'Edificio'
 'Casa uso de suelo', 'Local en centro comercial', 'Bodega comercial',
 'Otros', 'Villa', 'Duplex', 'Inmuebles productivos urbanos',
 'Departamento Compartido', 'Nave industrial', 'Rancho', 'Terreno industrial','Huerta', 'Lote', 'Hospedaje', 'Garage']

categories_list = provincias_list + ciudades_list + tipodepropiedad_list
#print(df_train_xgboost.tipodepropiedad.unique())
#enc = OneHotEncoder(categories = categories_list, handle_unknown = 'ignore')
df_train_xgboost = df_train
df_train_xgboost = df_train_xgboost.drop(['fecha','titulo','descripcion','direccion','idzona','lat','lng'], axis=1)
df_train_xgboost = df_train_xgboost.fillna(value = {'ciudad': 0, 'provincia': 0, 'tipodepropiedad': 0})


#ct = ColumnTransformer([(OneHotEncoder(categories = categories_list),['ciudad','provincia','tipodepropiedad'])])


print(df_train_xgboost.shape)
ct.fit_transform(df_train_xgboost)
print(df_train_xgboost.shape)

(240000, 16)


NameError: name 'ct' is not defined

In [7]:
#----------------PREPROCESAMIENTO----------------
df_train_xgboost = df_train
df_test_xgboost = df_test

#df_train_xgboost = df_train_xgboost.drop(['fecha','titulo','descripcion','direccion','idzona','lat','lng'], axis=1)
#df_train_xgboost = df_train_xgboost.fillna(value = {'ciudad': 0, 'provincia': 0, 'tipodepropiedad': 0})

df_train_xgboost = df_train_xgboost.drop(['ciudad','fecha','titulo','descripcion','direccion','idzona','lat','lng'], axis=1)
df_test_xgboost = df_test_xgboost.drop(['ciudad','fecha','titulo','descripcion','direccion','idzona','lat','lng'], axis=1)
#df_train_xgboost = df_train_xgboost.fillna(value = {'provincia': 0, 'tipodepropiedad': 0})

print(df_train_xgboost.shape)
#XGBoost maneja los NaNs, ver despues si lo dejamos como esta

df_train_xgboost = pd.get_dummies(df_train_xgboost, drop_first=True)#, sparse = True)
df_test_xgboost = pd.get_dummies(df_test_xgboost, drop_first=True)#, sparse = True)

print(df_train_xgboost.shape)
precio = df_train_xgboost['precio']
df_train_xgboost.drop(labels=['precio'], axis=1,inplace = True)
df_train_xgboost.insert(len(df_train_xgboost.columns), 'Precio', precio)

X_train, y_train = df_train_xgboost.iloc[:,:-1],df_train_xgboost.iloc[:,-1]

df_test_xgboost['tipodepropiedad_Garage'] = 0
df_test_xgboost['tipodepropiedad_Hospedaje'] = 0
df_test_xgboost = df_test_xgboost[list(X_train.columns.values)]

#data_dmatrix = xgb.DMatrix(data=X,label=y)
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

#xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 1, learning_rate = 0.01,
#                max_depth = 3, gamma = 1, n_estimators = 1000)

#xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.7, learning_rate = 0.05,
#                max_depth = 7, min_child_weight = 4, n_estimators = 500, nthread = 4, subsample = 0.7, silent = 1)

xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.7, learning_rate = 0.04,
                max_depth = 9, min_child_weight = 1, n_estimators = 700, nthread = 4, subsample = 0.7, silent = 1)

xg_reg.fit(X_train,y_train)

preds = xg_reg.predict(df_test_xgboost)


res = pd.DataFrame(preds, index=df_test_xgboost.index, columns=['precio'])


(240000, 15)
(240000, 67)


In [8]:
res2 = res
res2['id'] = df_test_xgboost['id']
res = res.reset_index(drop=True)
res.head()
res.set_index('id', inplace = True)
#res.columns = ['id', 'target']
#res.set_index('id', inplace = True)
res.rename(columns={'precio':'target'}, inplace=True)
res.head()


res.to_csv("XGBoost_grupo3.csv", header=True)